In [ ]:

# imports
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import minmax_scale
import seaborn as sns
from sklearn.model_selection import KFold

%matplotlib inline
plt.style.use('dark_background')

%load_ext autoreload

%autoreload 2

In [ ]:
root_folder = Path.cwd().parent.parent
data_folder = Path(root_folder).joinpath('data', 'external')

In [ ]:
from src.data.task_1_food_security.dataset import Dataset

In [ ]:
ds = Dataset(root_folder=root_folder)

ds.prepare_dataset_array()

In [ ]:
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

def create_new_model():
    model = Sequential()
    model.add(LSTM(126, input_shape=(36, 11)))
    model.add(Dense(1, activation='linear'))
    return model

In [ ]:
model = create_new_model()
model.summary()


In [ ]:
VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

save_dir = Path.cwd().joinpath('saved_models')
fold_var = 1

kf = KFold(n_splits = 5)
history_dir = {}
for train_index, val_index in kf.split(ds.X_train):
    training_x = ds.X_train[train_index]
    training_y = ds.y_train[train_index]
    validation_x = ds.X_train[val_index]
    validation_y = ds.y_train[val_index]

    # CREATE NEW MODEL
    model = create_new_model()

    # COMPILE NEW MODEL
    adam = Adam(lr=0.001)
    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                  optimizer=adam,
                  metrics=[tf.keras.losses.MeanSquaredError()])

    # CREATE CALLBACKS
    chk = ModelCheckpoint(f'saved_models/best_model{fold_var}.pkl',
                      monitor='mean_squared_error',
                      save_best_only=True,
                      mode='min',
                      verbose=0)

    callbacks_list = [chk]
    # There can be other callbacks, but just showing one because it involves the model name
    # This saves the best model
    # FIT THE MODEL

    history = model.fit(training_x,
                        training_y,
                        epochs=100,
                        batch_size=100,
                        callbacks=callbacks_list,
                        validation_data=(validation_x,validation_y),
                        verbose=0)

    #PLOT HISTORY
    #		:
    #		:

    # LOAD BEST MODEL to evaluate the performance of the model
    model.load_weights(f'saved_models/best_model{fold_var}.pkl')

    results = model.evaluate(validation_x, validation_y)
    results = dict(zip(model.metrics_names,results))
    history_dir[str(fold_var)] = history
    VALIDATION_ACCURACY.append(results['mean_squared_error'])
    VALIDATION_LOSS.append(results['loss'])
    print(f"Fold {fold_var}")
    print(results)
    print("+++++++++++++"*5)
    tf.keras.backend.clear_session()

    fold_var += 1



In [ ]:
p = {'first_neuron':[9,10,11],
     'hidden_layers':[0, 1, 2],
     'batch_size': [30],
     'epochs': [100],
     'dropout': [0],
     'kernel_initializer': ['uniform','normal'],
     'optimizer': ['Nadam', 'Adam'],
     'losses': ['binary_crossentropy'],
     'activation':['relu', 'elu'],
     'last_activation': ['sigmoid']}

In [ ]:

# and run the experiment
t = talos.Scan(x=x,
               y=y,
               model=breast_cancer_model,
               params=p,
               experiment_name='breast_cancer',
               round_limit=10)
